In [1]:
import torch
from model_triton import t_indexing_bkwd2

BS, N, D, V = 8, 512, 768, 4377
dloss_dx = torch.randn((BS, N, D), device="cuda")
layer_params0 = torch.randn((V, D), device="cuda")
x = torch.randint(V, (BS, N), device="cuda")

res1 = t_indexing_bkwd2(dloss_dx, (layer_params0, ), x)
res1


(tensor([[ 0.8921,  1.6637,  2.3464,  ..., -1.3423,  2.0934, -3.0515],
         [ 2.2695,  0.1287,  1.8906,  ...,  0.0890, -1.7803,  0.2788],
         [ 0.5469,  1.2861,  0.1816,  ...,  1.1836, -0.5928,  0.1455],
         ...,
         [ 0.0288, -0.5015,  0.6064,  ..., -0.2993, -0.3796,  0.0148],
         [-2.4434,  2.9639, -3.3328,  ...,  1.2504,  0.1337,  3.9976],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
        device='cuda:0'),)

In [2]:
def n_t_indexing_bkwd2(dloss_dx, layer_params, x, coef=1):
    x_1d = x.reshape(-1)
    D = dloss_dx.shape[-1]
    dloss_dx_2d = dloss_dx.reshape((-1, D))
    
    output = torch.zeros(layer_params[0].shape, device=x.device)
    indices = x_1d.unsqueeze(1).expand(x_1d.shape[0], D).to(torch.int64) # weirdly I need this expand here
    output.scatter_add_(0, indices,  dloss_dx_2d)
    return (coef*output, )

res2 = n_t_indexing_bkwd2(dloss_dx, (layer_params0, ), x)

In [3]:
assert torch.allclose(res1[0], res2[0], atol=1e-2, rtol=0), (res1[0].shape, res2[0].shape, res1[0][:10,:10], res2[0][:10,:10])
assert torch.allclose(res1[0], res2[0], atol=1e-3, rtol=0), "atol=1e-3 not working"

AssertionError: atol=1e-3 not working